In [73]:
import pandas as pd
import glob
import pickle

In [74]:
df = pd.read_csv('Results/DataFrame_clustering.csv', index_col=0)
df

,agg_idx,set_inds,-eigeval,curr_pt,Xarr_inds,dim_act
0,294,"[1, 9, 53]",0.630901,"(0.5, 0.5, 0.5)","[70, 78, 122, 594, 638, 2332]",3
1,1457,"[6, 10, 27]",0.630901,"(0.5, 0.5, 0.5)","[405, 409, 426, 655, 672, 1539]",3
2,1358,"[5, 30, 53]",0.630901,"(0.5, 0.5, 0.5)","[340, 365, 388, 1665, 1688, 2332]",3
3,983,"[4, 9, 28]",0.724745,"(0.5, 0.5, 0.5)","[274, 279, 298, 594, 613, 1582]",3
4,3190,"[14, 30, 52]",0.630901,"(0.5, 0.5, 0.5)","[889, 905, 927, 1665, 1687, 2314]",3
...,...,...,...,...,...,...
95,5026,"[27, 59, 65]",0.010642,"(0.3949700870039282, 0.7549270345915515, 0.214...","[1539, 1571, 1577, 2419, 2425, 2470]",3
96,1358,"[5, 30, 53]",0.035928,"(0.2811113388216673, 0.8327648376640984, 0.694...","[340, 365, 388, 1665, 1688, 2332]",3
97,1987,"[7, 55, 59]",0.007475,"(0.509513721416777, 0.1420472310178985, 0.7549...","[469, 517, 521, 2365, 2369, 2419]",3
98,2312,"[9, 36, 42]",0.013064,"(0.6705761595366357, 0.42109755589871434, 0.70...","[594, 621, 627, 1890, 1896, 2079]",3


In [129]:
def _read_rank_list(filename):
    with open(filename, 'rb') as f:
        rank_list = pickle.load(f)
        
    rank_list = pd.DataFrame(rank_list)
    return rank_list

# Get all CSV files in the folder starting with "rank"
pickle_files = glob.glob("temp_files/rank_list_*.pickle")
# Concatenate all DataFrames into a single DataFrame
rank_list = pd.concat(_read_rank_list(f) for f in pickle_files)

rank_list.reset_index(inplace=True, drop=True)
rank_list['cut_round'] = (rank_list.index // 100) + 1 

print(rank_list.shape)

unique_tripltes = pd.DataFrame(rank_list[1].value_counts()).reset_index()
unique_tripltes.rename(columns={1: 'count', 'index':'triplet'}, inplace=True)

rounds = []
for row in range(len(unique_tripltes)):
    triplet = str(unique_tripltes['triplet'].iloc[row])
    rounds.append(rank_list[rank_list[1].astype(str)==triplet]['cut_round'].values)
    
unique_tripltes['rounds']=rounds
unique_tripltes

(1200, 7)


,triplet,count,rounds
0,"[1, 35, 60]",3,"[3, 6, 12]"
1,"[24, 31, 59]",2,"[2, 11]"
2,"[6, 35, 68]",2,"[2, 4]"
3,"[11, 31, 60]",2,"[9, 10]"
4,"[3, 31, 52]",2,"[1, 2]"
...,...,...,...
1166,"[4, 56, 61]",1,[5]
1167,"[4, 61, 62]",1,[5]
1168,"[27, 41, 53]",1,[5]
1169,"[14, 17, 19]",1,[5]


In [158]:
df.head(2)

,0,1,2,3,4,5,prev_selected
0,0,"[0, 2, 9]",0.003574,"(0.6301528327749935, 0.6426488233393189, 0.602...","[0, 2, 9, 139, 146, 594]",-6.968432,0
1,1,"[0, 2, 16]",0.056513,"(0.6301528327749935, 0.6426488233393189, 0.547...","[0, 2, 16, 139, 153, 1000]",-3.700533,0


In [173]:
df = pd.read_csv('temp_files/data_main_20.csv', index_col=0)

df['prev_selected'] = df['1'].astype(str).isin(unique_tripltes['triplet'].astype(str)).astype(int)

selected_rows = df['prev_selected'] == 1
triplets = df.loc[selected_rows, '1'].astype(str)
matching_counts = unique_tripltes.loc[unique_tripltes['triplet'].astype(str).isin(triplets), 'count'].values
df.loc[selected_rows, 'prev_selected'] = df.loc[selected_rows, 'prev_selected'] * matching_counts
df

,0,1,2,3,4,5,prev_selected
0,2,"[0, 2, 17]",0.020681,"(0.6225299060456687, 0.6665981414595601, 0.168...","[0, 2, 17, 139, 154, 1054]",-2.977479,0
1,3,"[0, 2, 24]",0.003956,"(0.6225299060456687, 0.6665981414595601, 0.684...","[0, 2, 24, 139, 161, 1404]",-4.178744,0
2,5,"[0, 2, 39]",0.022474,"(0.6225299060456687, 0.6665981414595601, 0.656...","[0, 2, 39, 139, 176, 1989]",-0.625578,0
3,6,"[0, 2, 42]",0.001207,"(0.6225299060456687, 0.6665981414595601, 0.702...","[0, 2, 42, 139, 179, 2079]",-1.240100,0
4,13,"[0, 8, 37]",0.001765,"(0.6225299060456687, 0.2868184115774006, 0.645...","[0, 8, 37, 532, 561, 1924]",-4.563306,0
...,...,...,...,...,...,...,...
1525,6595,"[60, 62, 67]",0.019959,"(0.35017086953158183, 0.6447341481349177, 0.58...","[2430, 2432, 2437, 2449, 2454, 2479]",-3.974843,1
1526,6597,"[61, 65, 68]",0.018605,"(0.5925335382698518, 0.21485712039117838, 0.60...","[2440, 2444, 2447, 2470, 2473, 2482]",-3.306477,0
1527,6601,"[62, 65, 66]",0.001539,"(0.6447341481349177, 0.21485712039117838, 0.82...","[2449, 2452, 2453, 2470, 2471, 2475]",-3.608158,0
1528,6603,"[63, 64, 68]",0.048250,"(0.392497563510192, 0.29872262525008497, 0.608...","[2457, 2458, 2462, 2464, 2468, 2482]",-0.433125,0
